In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

## Scrap the data from wikipedia

In [20]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_Houston_neighborhoods')
houston_data = df[0]

In [21]:
# houston_data.columns = houston_data.loc[0,]
houston_data = houston_data.drop(columns=['#'])
houston_data.head()

,Name,Location relative to Downtown Houston,Approximate boundaries
0,Willowbrook,Northwest,Along Texas State Highway 249 northwest of Bel...
1,Greater Greenspoint,North,Around the junction of Beltway 8 and Interstat...
2,Carverdale,Northwest,South of the junction of Beltway 8 and U.S. Ro...
3,Fairbanks / Northwest Crossing,Northwest,Along U.S. Route 290 between Interstate 610 an...
4,Greater Inwood,Northwest,North of Fairbanks / Northwest Crossing and ea...
5,Acres Home,Northwest,West of Interstate 45 North and south of State...
6,Hidden Valley,North,In a triangular area between Veterans Memorial...
7,Westbranch,West,Along Beltway 8 south of Jersey Village
8,Addicks / Park Ten,West,Includes the entirety of Addicks Reservoir and...
9,Spring Branch West,West,North of Interstate 10 west between Addicks Re...


## Data Wrangling

In [22]:
houston_data.drop([0], axis=0, inplace=True)
houston_data.drop(columns=['Approximate boundaries'], inplace=True)
houston_data.columns = ['Neighborhood', 'Relative_Location']

In [23]:
houston_data['Relative_Location'] = houston_data['Relative_Location'].apply(lambda x: x+', Houston')
houston_data.head()

,Neighborhood,Relative_Location
1,Greater Greenspoint,"North, Houston"
2,Carverdale,"Northwest, Houston"
3,Fairbanks / Northwest Crossing,"Northwest, Houston"
4,Greater Inwood,"Northwest, Houston"
5,Acres Home,"Northwest, Houston"


In [24]:
#!pip install geoPy

## Convert addresses into Latitude and Longitude

In [25]:
from geopy.geocoders import Nominatim
lat=[]
lng=[]
def getLatLng(row):
    geolocator = Nominatim(user_agent='foursquare')
    print(row[0]+', '+row[1])
    location = geolocator.geocode(row[0]+', '+row[1])
    if location != None:
        lat.append(location.latitude)
        lng.append(location.longitude)
    else:
        lat.append(None)
        lng.append(None)


In [26]:
houston_data.apply(getLatLng, axis=1)
houston_data.head()

Greater Greenspoint, North, Houston
Carverdale, Northwest, Houston
Fairbanks / Northwest Crossing, Northwest, Houston
Greater Inwood, Northwest, Houston
Acres Home, Northwest, Houston
Hidden Valley, North, Houston
Westbranch, West, Houston
Addicks / Park Ten, West, Houston
Spring Branch West, West, Houston
Langwood, Northwest, Houston
Central Northwest (formerly Near Northwest), Northwest, Houston
Independence Heights, North, Houston
Lazybrook / Timbergrove, Northwest, Houston
Greater Greenspoint, North, Houston
Carverdale, Northwest, Houston
Fairbanks / Northwest Crossing, Northwest, Houston
Greater Inwood, Northwest, Houston
Acres Home, Northwest, Houston
Hidden Valley, North, Houston
Westbranch, West, Houston
Addicks / Park Ten, West, Houston
Spring Branch West, West, Houston
Langwood, Northwest, Houston
Central Northwest (formerly Near Northwest), Northwest, Houston
Independence Heights, North, Houston
Lazybrook / Timbergrove, Northwest, Houston
Greater Heights, Northwest, Houston


GeocoderTimedOut: ('Service timed out', 'occurred at index 40')

In [ ]:
houston_data['Latitude']=lat
houston_data['Longitude']=lng

In [ ]:
houston_data.shape

In [ ]:
houston_data.head()

In [ ]:
houston_data.dropna(axis=0, inplace=True)

In [ ]:
houston_data.reset_index(drop=True)
houston_data.head()

In [ ]:
!pip install folium
import folium

In [ ]:
hlat, hlng = 29.7604,-95.3698
houston_map = folium.Map([hlat, hlng], zoom_start=10)
houston_map

In [ ]:
for label, lat, lng in zip(houston_data['Neighborhood'], houston_data['Latitude'], houston_data['Longitude']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, lng],
        radius=10,
        popup=label,
        colur='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False
    ).add_to(houston_map)
    
houston_map


## Visuilize neighborhood of Houston using folium

In [ ]:
CLIENT_ID = '2TIYKHEA3JWL0PEZHBDCSOTPKYNH3NPG1XUAOC4S2COIVOY2' # your Foursquare ID
CLIENT_SECRET = 'GQVR1LLW0H13K1CEWXNYXHY4TASNA2TDKYP0WROZ002OQGUM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
LIMIT=100

In [ ]:
def getNearbyRestaurants(names, latitudes, longitudes, radius=3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&section=food&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
houston_restaurants = getNearbyRestaurants(names=houston_data['Neighborhood'],
                                   latitudes=houston_data['Latitude'],
                                   longitudes=houston_data['Longitude']
                                  )

## Exploring Unique Venues

In [ ]:
print(houston_restaurants.shape)
len(houston_restaurants['Venue Latitude'].unique())

In [ ]:
houston_restaurants_unique = houston_restaurants.drop_duplicates(subset=['Venue Latitude', 'Venue Longitude'], keep='first')

In [ ]:
houston_restaurants_unique[houston_restaurants_unique['Venue Category'].str.contains('Indian')].head()

## Visualize all the venues in Houston

In [ ]:
hlat, hlng = 29.7604,-95.3698
rest_map = folium.Map([hlat, hlng], zoom_start=10)
for label, lat, lng in zip(houston_restaurants_unique['Venue'], houston_restaurants_unique['Venue Latitude'], 
                           houston_restaurants_unique['Venue Longitude']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        popup=label,
        colur='red',
        fill=True,
        fill_color='green',
        fill_opacity=0.6,
        parse_html=False
    ).add_to(rest_map)
from IPython.display import display
display(rest_map)

In [ ]:
houston_restaurants_unique.groupby('Neighborhood').count()

In [ ]:
houston_onehot = pd.get_dummies(houston_restaurants_unique['Venue Category'])
houston_onehot.insert(loc=0, column='Neighborhood', value=houston_restaurants_unique['Neighborhood'])

In [ ]:
houston_onehot.head()

In [ ]:
houston_grouped = houston_onehot.groupby('Neighborhood').mean().reset_index()

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = houston_grouped['Neighborhood']

for ind in np.arange(houston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(houston_grouped.iloc[ind, :], num_top_venues)



## Top 10 Venues in each neighborhood

In [ ]:
neighborhoods_venues_sorted

## Clustering Neighborhoods

In [ ]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

houston_grouped_clustering = houston_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(houston_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

houston_merged = houston_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
houston_merged = houston_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
houston_merged.head() # check the last columns!

In [ ]:
houston_merged['Cluster Labels'] = houston_merged['Cluster Labels'].fillna(0)
houston_merged['Cluster Labels'] =   houston_merged['Cluster Labels'].astype(int)
houston_merged.head()

In [ ]:
# create map
map_clusters = folium.Map(location=[hlat, hlng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(houston_merged['Latitude'], houston_merged['Longitude'], houston_merged['Neighborhood'], houston_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the Clusters

In [ ]:
houston_merged.loc[houston_merged['Cluster Labels'] == 0, houston_merged.columns[[0] + list(range(5, houston_merged.shape[1]))]]
houston_merged.head()

In [ ]:
houston_merged.loc[houston_merged['Cluster Labels'] == 1, houston_merged.columns[[0] + list(range(5, houston_merged.shape[1]))]]
houston_merged.head()

In [ ]:
houston_merged.loc[houston_merged['Cluster Labels'] == 2, houston_merged.columns[[0] + list(range(5, houston_merged.shape[1]))]]

In [ ]:
houston_merged.loc[houston_merged['Cluster Labels'] == 3, houston_merged.columns[[0] + list(range(5, houston_merged.shape[1]))]]

In [ ]:
houston_merged.loc[houston_merged['Cluster Labels'] == 4, houston_merged.columns[[0] + list(range(5, houston_merged.shape[1]))]]